In [19]:
from sklearn.cluster import *
from sklearn import metrics
from sklearn.mixture import GaussianMixture  # 高斯混合模型
import os
import numpy as np
import librosa
import IPython.display as ipd
from glob import glob
from tqdm import tqdm
import pandas as pd

In [38]:
label_file = "./data/Angelic_Chaos_RE-BOOT/esd.list"
label = pd.read_csv(label_file, sep="|", header=None)
wavnames = label[0].tolist()
emb_paths = [i.replace(".wav", ".emo.npy") for i in wavnames]
embs = []
for embpath in tqdm(emb_paths):
    embs.append(np.expand_dims(np.load(embpath), axis=0))

100%|██████████| 25618/25618 [01:00<00:00, 425.70it/s]


In [39]:
x = np.concatenate(embs, axis=0)

# 聚类算法类的数量
n_clusters = 3
model = KMeans(n_clusters=n_clusters, random_state=10)
# 可以自行尝试各种不同的聚类算法
# model = Birch(n_clusters= n_clusters, threshold= 0.2)
# model = SpectralClustering(n_clusters=n_clusters)
# model = AgglomerativeClustering(n_clusters= n_clusters)

y_predict = model.fit_predict(x)

/home/oedosoldier/.conda/envs/tts/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [40]:
def disp(wavname):
    wav, sr = librosa.load(wavname, 16000)
    display(ipd.Audio(wav, rate=sr))


# 获取预测结果各类的文件名
classes = {"wavname": [], "class": []}
for i in range(len(y_predict)):
    classes["wavname"].append(wavnames[i])
    classes["class"].append(y_predict[i])
classes = pd.DataFrame.from_dict(classes)

# 每类随机选取 4 个样本
samples = classes.groupby("class").apply(lambda x: x.sample(4))
samples = samples.reset_index(drop=True)

# 展示每类音频
for i in range(n_clusters):
    print("类别:", i, "本类中样本数量:", len(classes[classes["class"] == i]))
    for wavname in samples[samples["class"] == i]["wavname"]:
        print(wavname)
        disp(wavname)

类别: 0 本类中样本数量: 7390
./data/Angelic_Chaos_RE-BOOT/wavs/kur_315_0203.wav


/tmp/ipykernel_190037/1152397497.py:2: FutureWarning: Pass sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav, sr = librosa.load(wavname, 16000)


./data/Angelic_Chaos_RE-BOOT/wavs/ori_601_0010.wav


./data/Angelic_Chaos_RE-BOOT/wavs/noa_106_0057.wav


./data/Angelic_Chaos_RE-BOOT/wavs/kur_006_0161.wav


...

./data/Angelic_Chaos_RE-BOOT/wavs/noa_108_0128.wav


类别: 2 本类中样本数量: 12345
./data/Angelic_Chaos_RE-BOOT/wavs/ama_203_0046.wav


./data/Angelic_Chaos_RE-BOOT/wavs/fum_603_0066.wav


./data/Angelic_Chaos_RE-BOOT/wavs/ama_315_0002.wav


./data/Angelic_Chaos_RE-BOOT/wavs/kur_009_0051.wav
